# Loading Data

In [1]:
from datasets import load_dataset, load_from_disk

tydiqa_gold = load_dataset("khalidalt/tydiqa-goldp", 'indonesian', trust_remote_code=True)
mr_tydi = load_from_disk("./generated_data/mr_tydi_filled")

In [2]:
tydiqa_gold

DatasetDict({
    train: Dataset({
        features: ['id', 'language', 'document_title', 'passage_text', 'question_text', 'answers'],
        num_rows: 5702
    })
    validation: Dataset({
        features: ['id', 'language', 'document_title', 'passage_text', 'question_text', 'answers'],
        num_rows: 565
    })
})

# Data Cleaning

In [3]:
tydiqa_gold = tydiqa_gold.remove_columns(["language", "document_title", "passage_text"])
tydiqa_gold = tydiqa_gold.rename_column("id", "tydiqa_id")
tydiqa_gold = tydiqa_gold.rename_column("question_text", "query")

In [4]:
from datasets import DatasetDict

def clean_answers(example):
    example["answers"] = example["answers"]["text"]  # Ambil hanya bagian text
    return example

# Terapkan fungsi untuk membersihkan answers di setiap split
tydiqa_gold = DatasetDict({
    split: dataset.map(clean_answers)
    for split, dataset in tydiqa_gold.items()
})

In [5]:
import re
from datasets import load_from_disk, DatasetDict

# Fungsi untuk membersihkan teks: hapus newline & whitespace berlebih
def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

# Fungsi untuk membersihkan dan memilih jawaban terpendek
def clean_tydiqa(example):
    # Bersihkan query
    example["query"] = clean_text(example["query"])
    
    # Bersihkan answers dan pilih jawaban terpendek jika ada lebih dari satu
    cleaned_answers = [clean_text(ans) for ans in example["answers"]]
    example["answers"] = min(cleaned_answers, key=len) if cleaned_answers else ""  # Pilih jawaban terpendek

    return example

# Fungsi untuk membersihkan query di MR-TyDi
def clean_mr_tydi(example):
    example["query"] = clean_text(example["query"])
    return example

# Terapkan pembersihan pada dataset
tydiqa_gold_cleaned = DatasetDict({
    split: dataset.map(clean_tydiqa)
    for split, dataset in tydiqa_gold.items()
})

mr_tydi_cleaned = DatasetDict({
    split: dataset.map(clean_mr_tydi)
    for split, dataset in mr_tydi.items()
})

Map:   0%|          | 0/5702 [00:00<?, ? examples/s]

Map:   0%|          | 0/565 [00:00<?, ? examples/s]

In [6]:
check_if_answer_is_cleaned = tydiqa_gold_cleaned['train'].filter(lambda x: x['tydiqa_id']=="-2253919563477221294-3")
print(check_if_answer_is_cleaned[0]['answers'])

check_if_answer_more_than_1 = tydiqa_gold_cleaned['validation'].filter(lambda x: x['tydiqa_id']=="8601389648636013237-1")
check_if_answer_more_than_1[0]

Filter:   0%|          | 0/5702 [00:00<?, ? examples/s]

Nicolas Loufrani


Filter:   0%|          | 0/565 [00:00<?, ? examples/s]

{'tydiqa_id': '8601389648636013237-1',
 'query': 'siapakah ketua Perum LKBN pertama?',
 'answers': 'Mr. Soemanang'}

In [46]:
import re
from collections import Counter
from datasets import concatenate_datasets

# Gabungkan split train dan validation
tydiqa_gold_combined = concatenate_datasets([tydiqa_gold_cleaned["train"], tydiqa_gold_cleaned["validation"]])

# Fungsi untuk menghitung jumlah kata dalam jawaban
def count_words(text):
    return len(re.findall(r'\w+', text))

# Filter hanya row dengan answers lebih dari 10 kata
long_answer_rows = [row for row in tydiqa_gold_combined if count_words(row["answers"]) > 10]

# Ambil kata pertama dari query
first_words = [row["query"].split()[0] for row in long_answer_rows]

# Hitung jumlah kemunculan kata pertama dalam query
first_word_counts = Counter(first_words)

# Konversi ke daftar kata unik
unique_first_words = sorted(set(first_words))

# Cetak hasil
print(f"🔍 Ditemukan {len(unique_first_words)} kata unik yang menjadi kata pertama dalam query ketika answers memiliki lebih dari 10 kata.")
print("📋 Kata unik tersebut:", unique_first_words)


🔍 Ditemukan 42 kata unik yang menjadi kata pertama dalam query ketika answers memiliki lebih dari 10 kata.
📋 Kata unik tersebut: ['Aapakah', 'Ada', 'Apa', 'Apaka', 'Apakah', 'Bagaimana', 'Bagaimanakah', 'Berapa', 'Berapakah', 'Bidang', 'Dari', 'Darimanakah', 'Daun', 'Di', 'Dimana', 'Dimanakah', 'Dimankah', 'Kapan', 'Kenapa', 'Menceritakan', 'Mengapa', 'Musik', 'Siapa', 'Siapakah', 'Tentang', 'Untuk', 'Zat', 'aoakah', 'apa', 'apaah', 'apakah', 'berapa', 'berapakah', 'berdasarkan', 'dari', 'di', 'dimanakah', 'kapankah', 'mengapa', 'negara', 'siapakah', 'siapakh']


# Filtering untuk answers yang jumlah katanya kurang dari 15 (opsional)

# Join

In [57]:
tydiqa_gold_cleaned

DatasetDict({
    train: Dataset({
        features: ['tydiqa_id', 'query', 'answers'],
        num_rows: 5702
    })
    validation: Dataset({
        features: ['tydiqa_id', 'query', 'answers'],
        num_rows: 565
    })
})

In [7]:
from datasets import DatasetDict, concatenate_datasets

# Gabungkan split train & validation pada tydiqa_gold_cleaned
tydiqa_gold_combined = concatenate_datasets([tydiqa_gold_cleaned["train"], tydiqa_gold_cleaned["validation"]])

# Buat struktur baru mengikuti split dari mr_tydi_cleaned
joined_datasets = {}

for split, mr_tydi_split in mr_tydi_cleaned.items():
    # Buat dictionary {query: row} dari tydiqa_gold_cleaned untuk lookup cepat
    tydiqa_gold_dict = {row["query"]: row for row in tydiqa_gold_combined}
    
    # Buat daftar baru dengan menggabungkan informasi dari mr_tydi_cleaned dan tydiqa_gold_cleaned
    new_split_data = []
    
    for row in mr_tydi_split:
        query = row["query"]
        tydiqa_data = tydiqa_gold_dict.get(query, None)  # Ambil data dari tydiqa_gold jika ada
        
        # Gabungkan data (jika tidak ada di tydiqa_gold, biarkan bagian tersebut kosong)
        merged_row = {
            **row,  # Data dari mr_tydi_cleaned
            "tydiqa_id": tydiqa_data["tydiqa_id"] if tydiqa_data else None,
            "answers": tydiqa_data["answers"] if tydiqa_data else None
        }
        
        new_split_data.append(merged_row)

    # Konversi kembali ke Dataset
    joined_datasets[split] = mr_tydi_split.from_list(new_split_data)

# Simpan hasil sebagai DatasetDict
final_dataset = DatasetDict(joined_datasets)

# # Simpan hasil ke disk
# final_dataset.save_to_disk("generated_data/mr_tydi_tydiqa_joined")

print("✅ Dataset berhasil digabungkan berdasarkan `query` dengan struktur mengikuti `mr_tydi_cleaned`.")
# print("✅ Dataset telah disimpan di 'generated_data/mr_tydi_tydiqa_joined'.")


✅ Dataset berhasil digabungkan berdasarkan `query` dengan struktur mengikuti `mr_tydi_cleaned`.


In [9]:
from datasets import DatasetDict

# Fungsi untuk menghapus rows dengan answers = None
def remove_none_answers(dataset):
    return dataset.filter(lambda row: row["answers"] is not None)

# Buat dataset baru tanpa row yang memiliki answers = None
finished_dataset = DatasetDict({
    "train": remove_none_answers(final_dataset["train"]),
    "dev": remove_none_answers(final_dataset["dev"]),
    "test": remove_none_answers(final_dataset["test"])
})

print("✅ Semua row dengan 'answers = None' telah dihapus dari dataset baru `finished_dataset`.")

Filter:   0%|          | 0/4902 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1224 [00:00<?, ? examples/s]

Filter:   0%|          | 0/829 [00:00<?, ? examples/s]

✅ Semua row dengan 'answers = None' telah dihapus dari dataset baru `finished_dataset`.


In [10]:
# Fungsi untuk mengecek apakah ada None dalam kolom 'answers'
def check_none_answers(dataset, split_name):
    none_count = sum(1 for row in dataset if row["answers"] is None)
    print(f"🔍 Split '{split_name}': {none_count} row(s) memiliki 'answers' = None")

# Periksa setiap split
check_none_answers(finished_dataset["train"], "train")
check_none_answers(finished_dataset["dev"], "dev")
check_none_answers(finished_dataset["test"], "test")

🔍 Split 'train': 0 row(s) memiliki 'answers' = None
🔍 Split 'dev': 0 row(s) memiliki 'answers' = None
🔍 Split 'test': 0 row(s) memiliki 'answers' = None


In [16]:
from transformers import AutoTokenizer, AutoModel

# # Load dataset yang sudah dibersihkan
# finished_dataset = load_from_disk("generated_data/mr_tydi_tydiqa_cleaned")

# Load tokenizer & model untuk Multilingual-E5-Small
model_name = "intfloat/multilingual-e5-small"
embedding_tokenizer = AutoTokenizer.from_pretrained(model_name)
embedding_model = AutoModel.from_pretrained(model_name).to("cuda:0")

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [17]:
import torch.nn.functional as F
import torch
from torch import Tensor
from tqdm import tqdm
import gc
from datasets import load_from_disk


# Fungsi average pooling
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Fungsi untuk memilih top 4 negative_passages berdasarkan similarity
def select_top4_negative_passages(example):
    query_text = example["query"]
    negative_passages = example["negative_passages"]

    # Jika sudah <= 4, tidak perlu pemrosesan
    if len(negative_passages) <= 4:
        return example

    # Ambil teks dari negative_passages
    neg_texts = [neg["text"] for neg in negative_passages]

    # Tokenisasi dan embedding query serta negative_passages
    batch_dict = embedding_tokenizer([query_text] + neg_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    batch_dict = {k: v.to("cuda:0") for k, v in batch_dict.items()}

    with torch.no_grad():
        outputs = embedding_model(**batch_dict)

    # Hitung embedding dan normalisasi
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)  # Normalisasi untuk cosine similarity

    # Hitung similarity scores (query vs negative_passages)
    query_embedding = embeddings[0].unsqueeze(0)  # Query ada di indeks pertama
    neg_embeddings = embeddings[1:]  # Negative passages setelah query
    scores = (query_embedding @ neg_embeddings.T).squeeze(0)  # Cosine similarity

    # Ambil indeks top 4 dengan similarity tertinggi
    top_indices = torch.argsort(scores, descending=True)[:4]

    # Simpan hanya 4 negative_passages terbaik
    example["negative_passages"] = [negative_passages[i] for i in top_indices]

    # Bersihkan cache GPU setelah query diproses
    del batch_dict, outputs, embeddings, scores
    torch.cuda.empty_cache()
    gc.collect()

    return example

# Terapkan fungsi ke split train
finished_dataset["train"] = finished_dataset["train"].map(select_top4_negative_passages)

# Simpan hasilnya ke disk
finished_dataset.save_to_disk("generated_data/mr_tydi_tydiqa_final")

print("✅ Negative passages pada split train telah dipangkas menjadi top 4 berdasarkan similarity!")
print("✅ Dataset telah disimpan di 'generated_data/mr_tydi_tydiqa_final'.")


Map:   0%|          | 0/4542 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4542 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1143 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/565 [00:00<?, ? examples/s]

✅ Negative passages pada split train telah dipangkas menjadi top 4 berdasarkan similarity!
✅ Dataset telah disimpan di 'generated_data/mr_tydi_tydiqa_final'.
